In [ ]:
# Importing Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
# Reading csvs

animal_df = pd.read_csv('../input/doom-crossing/animal_crossing_dataset.csv')
doom_df = pd.read_csv('../input/doom-crossing/doom_crossing_dataset.csv')

In [ ]:
animal_df.head()

In [ ]:
doom_df.head()

In [ ]:
data_dir = '../input/doom-crossing/'

batch_size = 16
img_height = 224
img_width = 224

In [ ]:
# Loading training dataset

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.3,
  subset="training",
  seed=100,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
# Loading validation dataset

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=100,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
# Augmentation

train_dataset = train_ds
validation_dataset = val_ds

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
# Visualization

for image, _ in train_dataset.take(1):
    plt.figure(figsize=(10, 10))
    first_image = image[0]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
        plt.imshow(augmented_image[0] / 255)
        plt.axis('off')

In [ ]:
# Mobilenet Preprocessing

preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

In [ ]:
# Loading pretrained weights

weights = '../input/keras-pretrain-model-weights/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5'
IMG_SHAPE = (img_height, img_width) + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights=weights)

In [ ]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

In [ ]:
# Implementing layers on top of base model

base_model.trainable = False
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)
prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

In [ ]:
# Full Model

inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.3)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
# Compilation

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
# Training model

history = model.fit(train_dataset,
                    epochs=20,
                    validation_data=validation_dataset)


In [ ]:
# Plotting Accuracy Curves

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

## Exploration tasks

<b>Q-1 Which are the misclassified photos? How do you explain why they are misclassified? How can you help your model classify those photos correctly?</b>
 
Ans. The misclassified photos are mostly the ones which have text in them but don’t have appropriate features for Doom or Animal Crossing. They are misclassified because the model picks up on features and not the text, thus not appropriately classifying the pictures. We can run an additional OCR on the images to scan the text and can add a parallel pipeline to classify according to the text along with image features, giving text classification the greater priority.
 
 
<b>Q-2 How do you know if your model has actually learned to classify the images? Or is it simply very lucky at guessing?</b>
 
Ans. I have divided the whole dataset into training and validation. If the model is guessing then it should give a higher training accuracy but a lower validation accuracy, but since it's giving good accuracies at both seen and well as unseen data we can infer that it's actually learning.
 
 
<b>Q-3 What happens if you ask your model to classify a new image which is neither a Doom nor an Animal Crossing meme? You can try to upload a random photo and generate prediction on it to see the results. How would you modify your model to handle this situation?</b>
 
Ans. The model will classify the random photo into one of the categories regardless. The confidence score will surely be low. To handle this, we can create a third category ‘Not Doom or Animal Crossing’ and train the model again to handle the exceptions.
 
 
<b>Q-4 After having successfully trained a model to classify Doom and Animal Cross memes, you want your model to classify other memes as well, such as those from Genshin Impact. How would you avoid retraining the model from scratch?</b>
 
Ans.We can use the current as a base model and can train extra layers over it to classify other memes categories, ie again using the concept of Transfer Learning.

